In [1]:
import tensorflow as tf
import custom_loss.loss_of_scoreMap as scoreMap_loss
import siameFC.SiameFC as siameFc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=Warning)

from data_processing.data import make_label, load_images

In [2]:
Z_SHAPE = (127, 127, 3)
X_SHAPE = (255, 255, 3)
LABEL = (17, 17, 1)

x_dir = "./sample/"
z_dir = "./sample/"
x_images = load_images(x_dir, 255, 0, ".x.jpg")
z_images = load_images(z_dir, 127, 0, ".z.jpg")

In [3]:
radius = 16 #hyperparameter

response_size = 17 
response_stride = 8.0 #end siride = 9

data_size = 1

label = make_label(response_size, radius / response_stride)
labels = np.empty((data_size,) + label.shape)
labels[:] = label


model = siameFc.siameFc_model(X_SHAPE,Z_SHAPE)




Instructions for updating:
Use fn_output_signature instead


In [4]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_1 (InputLayer)           [(None, 127, 127, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 123, 123, 96  34944       ['input_2[0][0]']                
                                )                                                             

 rmalization)                                                                                     
                                                                                                  
 lambda (Lambda)                (None, 1, 17, 17, 1  0           ['batch_normalization_9[0][0]',  
                                )                                 'batch_normalization_4[0][0]']  
                                                                                                  
Total params: 3,794,432
Trainable params: 3,788,928
Non-trainable params: 5,504
__________________________________________________________________________________________________
None


In [ ]:
opt = tf.keras.optimizers.SGD(
            momentum=0.9, nesterov=False, name='SGD'
            )

model.compile(optimizer=opt, loss=scoreMap_loss, metrics=['accuracy'])


batch_size = 1
epochs = 1
model.fit([x_images, z_images], [labels], batch_size=batch_size, epochs=epochs)
